## LlamaParser
LlamaParse는 LlamaIndex에서 개발한 문서 파싱 서비스로, 대규모 언어 모델(LLM)을 위해 특별히 설계되었습니다. 주요 특징은 다음과 같습니다:

- PDF, Word, PowerPoint, Excel 등 다양한 문서 형식 지원
- 자연어 지시를 통한 맞춤형 출력 형식 제공
- 복잡한 표와 이미지 추출 기능
- JSON 모드 지원
- 외국어 지원

LlamaParse는 독립형 API로 제공되며, LlamaCloud 플랫폼의 일부로도 사용 가능합니다. 이 서비스는 문서를 파싱하고 정제하여 검색 증강 생성(RAG) 등 LLM 기반 애플리케이션의 성능을 향상시키는 것을 목표로 합니다.

사용자는 무료로 하루 1,000페이지를 처리할 수 있으며, 유료 플랜을 통해 추가 용량을 확보할 수 있습니다. LlamaParse는 현재 공개 베타 버전으로 제공되고 있으며, 지속적으로 기능이 확장되고 있습니다.

In [ ]:
# 설치
!pip install llama-index-core llama-parse llama-index-readers-file python-dotenv


In [ ]:
import os
import nest_asyncio
from dotenv import load_dotenv

LLAMA_CLOUD_API_KEY = ""
nest_asyncio.apply()


## Basic Para

In [ ]:
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader

# 파서 설정
parser = LlamaParse(
    result_type="markdown",  # "markdown"과 "text" 사용 가능
    num_workers=8,  # worker 수 (기본값: 4)
    verbose=True,
    language="ko",
    api_key=LLAMA_CLOUD_API_KEY,
)

# SimpleDirectoryReader를 사용하여 파일 파싱
file_extractor = {".pdf": parser}

# LlamaParse로 파일 파싱
documents = SimpleDirectoryReader(
    input_files=["/content/SPRI_AI_Brief_2023년12월호_F.pdf"],
    file_extractor=file_extractor,
).load_data()


In [ ]:
# 페이지 수 확인
len(documents)


In [ ]:
documents[0]

LlamaIndex -> LangChain Document 로 변환

In [ ]:
pip install -qU langchain_community

In [ ]:
# 랭체인 도큐먼트로 변환
docs = [doc.to_langchain_format() for doc in documents]


In [ ]:
!# metadata 출력
print(docs[5].page_content)


## MultiModal Model 로 파싱
주요 파라미터

- use_vendor_multimodal_model: 멀티모달 모델 사용 여부를 지정합니다. True로 설정하면 외부 벤더의 멀티모달 모델을 사용합니다.

- vendor_multimodal_model_name: 사용할 멀티모달 모델의 이름을 지정합니다. 여기서는 "openai-gpt4o"를 사용하고 있습니다.

- vendor_multimodal_api_key: 멀티모달 모델 API 키를 지정합니다. 환경 변수에서 OpenAI API 키를 가져옵니다.

- result_type: 파싱 결과의 형식을 지정합니다. "markdown"으로 설정되어 있어 결과가 마크다운 형식으로 반환됩니다.

- language: 파싱할 문서의 언어를 지정합니다. "ko"로 설정되어 한국어로 처리됩니다.

- skip_diagonal_text: 대각선 텍스트를 건너뛸지 여부를 결정합니다.

- page_separator: 페이지 구분자를 지정할 수 있습니다.

In [ ]:
OPENAI_API_KEY = ''
documents = LlamaParse(
    use_vendor_multimodal_model=True,
    vendor_multimodal_model_name="openai-gpt4o",
    vendor_multimodal_api_key=OPENAI_API_KEY,
    api_key=LLAMA_CLOUD_API_KEY,
    result_type="markdown",
    language="ko",
    # skip_diagonal_text=True,
    # page_separator="\n=================\n"
)

In [ ]:
# parsing 된 결과
parsed_docs = documents.load_data(file_path="data/SPRI_AI_Brief_2023년12월호_F.pdf")


In [ ]:
# langchain 도큐먼트로 변환
docs = [doc.to_langchain_format() for doc in parsed_docs]


In [ ]:
print(docs[0]. page_content)

In [ ]:
# parsing instruction 을 지정합니다.
parsing_instruction = (
    "You are parsing a brief of AI Report. Please extract tables in markdown format."
)

# LlamaParse 설정
parser = LlamaParse(
    use_vendor_multimodal_model=True,
    vendor_multimodal_model_name="openai-gpt4o",
    vendor_multimodal_api_key=OPENAI_API_KEY,
    result_type="markdown",
    language="ko",
    parsing_instruction=parsing_instruction,
    api_key=LLAMA_CLOUD_API_KEY,
)

# parsing 된 결과
parsed_docs = parser.load_data(file_path="/content/SPRI_AI_Brief_2023년12월호_F.pdf")

# langchain 도큐먼트로 변환
docs = [doc.to_langchain_format() for doc in parsed_docs]


In [ ]:
print(docs[-2]. page_content)